In [1]:
%pylab inline
import pandas as pd
import docplex.mp
from docplex.mp.model import Model

Populating the interactive namespace from numpy and matplotlib


In [2]:
raw_data=pd.read_excel('C:/Users/Ruosh/Desktop/717/assin/assin10/raw.xlsx',sheet_name='Sheet3', index_col=0)

In [3]:
Gen_list=list(raw_data.index)
Bus_list=['1','2','3']
Line_list=[('1','2'),('2','3'),('1','3')]

fuel_price={'NG':3.06, 'Coal': 2.16, 'Oil': 9.83} # $/mmBtu
carbon_price=0 # $/ton CO2
SO2_stan=0.7 # lbs/mmBtu
NOx_stan=0.15 # lbs/mmBtu
GHG_stan=600 # lbs/MWh

Load=1800

In [4]:
# dictionaries about each bus
bus1_gen=list((raw_data.loc[raw_data['Bus']==1]).index)
bus2_gen=list((raw_data.loc[raw_data['Bus']==2]).index)
bus3_gen=list((raw_data.loc[raw_data['Bus']==3]).index)
bus_dict={'1': bus1_gen, '2': bus2_gen, '3': bus3_gen}
load_dict={'1': 0.15, '2': 0.5, '3': 0.35}

# dictionaries about generators
fuel_dict=(raw_data.loc[:,'Fuel']).to_dict()
maxgen_dict=(raw_data.loc[:,'capacity(MW)']).to_dict() # MW
mingen_dict=(raw_data.loc[:,'Min Generation']).to_dict() # MW
HR_dict=(raw_data.loc[:,'Heat Rate']).to_dict() # Btu/kWh
NOx_dict=(raw_data.loc[:,'Nox']).to_dict() # lb/MWh
SO2_dict=(raw_data.loc[:,'SO2']).to_dict() # lb/MWh
CO2_dict=(raw_data.loc[:,'CO2']).to_dict() # lb/MWh
CH4_dict=(raw_data.loc[:,'CH4']).to_dict() # lb/MWh
N2O_dict=(raw_data.loc[:,'N2O']).to_dict() # lb/MWh

# dictionaries about three transmission lines
line_capacity_dict = dict(zip(Line_list, [1100, 2000, 2000]))
line_impedance = dict(zip(Line_list, [0.2,0.2,-0.2]))# here assume to all be 0.2/unit

In [5]:
md = Model(name='NicISO Operation')

In [6]:
# Decision Variables
## hourly generation of each generator (MW)
gens = md.continuous_var_dict(keys=Gen_list)
flows = md.continuous_var_dict(keys=[('1','2'),('2','3'),('1','3')])

In [7]:
# Constraints

## Hourly emission constraints
# md.add_constraint_(md.sum(gens[g] * 1 * SO2_dict[g] for g in Gen_list if fuel_dict[g]=='Coal') <= SO2_stan * md.sum(gens[g] * HR_dict[g] / 1000 for g in Gen_list if fuel_dict[g]=='Coal'), 'SO2 emission constraint')
# md.add_constraint_(md.sum(gens[g] * 1 * NOx_dict[g] for g in Gen_list if fuel_dict[g]=='Coal') <= NOx_stan * md.sum(gens[g] * HR_dict[g] / 1000 for g in Gen_list if fuel_dict[g]=='Coal'), 'NOx emission constraint')
# md.add_constraint_(md.sum(gens[g] * 1 * (CO2_dict[g] + CH4_dict[g] + N2O_dict[g]) for g in Gen_list) <= GHG_stan * md.sum(gens[g] for g in Gen_list), 'GHG emission constraint')

## KVL
md.add_constraint(md.sum(flows[l] * line_impedance[l] for l in Line_list) == 0, 'KVL along the loop')

## Bus balance
for b in Bus_list:
    md.add_constraint(md.sum(gens[g] for g in bus_dict[b]) - Load * load_dict[b] - md.sum(v for k,v in flows.items() if k[0]==b) + md.sum(v for k,v in flows.items() if k[1]==b) ==0, 'Bus balance for '+ b)

## Generator constraints
md.add_constraints_((gens[g] <= maxgen_dict[g] for g in Gen_list),'Max capacity')
md.add_constraints_((gens[g] >= mingen_dict[g] for g in Gen_list),'Min capacity')

## Supply meet demand
md.add_constraint_(md.sum(gens) == Load, 'Supply meets Demand')


## Line capacity
# md.add_constraints_(((flows[l]) <= line_capacity_dict[l] for l in Line_list), 'Max line capacity constraint')
# md.add_constraints_(((flows[l]) >= -1 * line_capacity_dict[l] for l in Line_list), 'Min line capacity constraint')

In [8]:
# Objective
coal_generation = md.sum(gens[g] for g in Gen_list if fuel_dict[g]=='Coal')
NG_generation = md.sum(gens[g] for g in Gen_list if fuel_dict[g]=='NG')
Oil_generation = md.sum(gens[g] for g in Gen_list if fuel_dict[g]=='Oil')
#co2_emission = md.sum(gens[g] * CO2_dict[g] for g in Gen_list)
coal_consumption = md.sum(gens[g] * HR_dict[g] / 1000 for g in Gen_list if fuel_dict[g]=='Coal')
NG_consumption = md.sum(gens[g] * HR_dict[g] / 1000 for g in Gen_list if fuel_dict[g]=='NG')
Oil_consumption = md.sum(gens[g] * HR_dict[g] / 1000 for g in Gen_list if fuel_dict[g]=='Oil')
md.minimize(coal_consumption * fuel_price['Coal'] + NG_consumption * fuel_price['NG'] + Oil_consumption * fuel_price['Oil'])
#+ carbon_price * co2_emission

In [9]:
md.solve()

docplex.mp.solution.SolveSolution(obj=57056.3,values={x1:519.2,x2:1051.2..

In [10]:
solution = md.solve()
solution.display()

solution for: NicISO Operation
objective: 57056.348
x1 = 519.200
x2 = 1051.200
x3 = 98.000
x4 = 4.000
x5 = 127.600
x7 = 249.200
x8 = 249.200


In [215]:
md.objective_expr

docplex.mp.LinearExpr(39.667x1+26.289x2+34.820x3+175.308x4+36.931x5)

In [216]:
md.is_optimized()

True

In [217]:
md.linear_constraint_basis_statuses(md.iter_linear_constraints())

[<BasisStatus.Basic: 1>,
 <BasisStatus.AtLowerBound: 0>,
 <BasisStatus.Basic: 1>,
 <BasisStatus.Basic: 1>,
 <BasisStatus.Basic: 1>,
 <BasisStatus.Basic: 1>,
 <BasisStatus.Basic: 1>,
 <BasisStatus.Basic: 1>,
 <BasisStatus.AtLowerBound: 0>,
 <BasisStatus.AtLowerBound: 0>,
 <BasisStatus.Basic: 1>,
 <BasisStatus.AtLowerBound: 0>,
 <BasisStatus.AtLowerBound: 0>,
 <BasisStatus.AtLowerBound: 0>,
 <BasisStatus.AtLowerBound: 0>]

In [218]:
md.number_of_constraints

15

In [219]:
for i in md.iter_variables():
    print(i)

x1
x2
x3
x4
x5
x6
x7
x8


In [220]:
for i in md.iter_linear_constraints():
    print(i)

Supply meets Demand: x1+x2+x3+x4+x5 == 1800
Max capacity1: x1 <= 519.2
Max capacity2: x2 <= 1848.0
Max capacity3: x3 <= 656.1
Max capacity4: x4 <= 27.0
Max capacity5: x5 <= 398.3
Min capacity1: x1 >= 78
Min capacity2: x2 >= 277
Min capacity3: x3 >= 98
Min capacity4: x4 >= 4
Min capacity5: x5 >= 60
Bus balance for 1: x1-x6-x8-270 == 0
Bus balance for 2: x2+x3+x6-x7-900 == 0
Bus balance for 3: x4+x5+x7+x8-630 == 0
KVL along the loop: 0.200x6+0.200x7-0.200x8 == 0
